In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from datasets import UnsupervisedLunarSeismicDataset, collate_fn, train_test_split_dataset
from models import CNNAutoencoder

In [6]:
# Define model and training hyperparameters
num_epochs = 30
batch_size=16
learning_rate=1e-3
data_dir = 'data/lunar/unsupervised'

In [7]:
# Load the dataset
unsupervised_dataset = UnsupervisedLunarSeismicDataset(data_dir=data_dir)
train_dataset, test_dataset = train_test_split_dataset(unsupervised_dataset)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

/root/projects/space_apps_2024_seismic_detection/datasets.py:19: UserWarning: nperseg = 256 is greater than input length  = 33, using nperseg = 33
  freq, time, spectrogram = signal.spectrogram(trace_filtered.data, trace_filtered.stats.sampling_rate)
/root/projects/space_apps_2024_seismic_detection/datasets.py:19: UserWarning: nperseg = 256 is greater than input length  = 22, using nperseg = 22
  freq, time, spectrogram = signal.spectrogram(trace_filtered.data, trace_filtered.stats.sampling_rate)
/root/projects/space_apps_2024_seismic_detection/datasets.py:19: UserWarning: nperseg = 256 is greater than input length  = 31, using nperseg = 31
  freq, time, spectrogram = signal.spectrogram(trace_filtered.data, trace_filtered.stats.sampling_rate)
/root/projects/space_apps_2024_seismic_detection/datasets.py:19: UserWarning: nperseg = 256 is greater than input length  = 11, using nperseg = 11
  freq, time, spectrogram = signal.spectrogram(trace_filtered.data, trace_filtered.stats.sampling_ra

In [8]:
# Init the model
autoencoder = CNNAutoencoder()

# Training set up
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=learning_rate)

In [9]:
# Training Loop
for epoch in range(num_epochs):
    autoencoder.train()
    for inputs in train_loader:
        if inputs is None:
            continue
        inputs = inputs.unsqueeze(1)  # Add channel dimension
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item():.4f}')

    # Evaluate on the test set
    autoencoder.eval()
    with torch.no_grad():
        test_loss = 0.0
        for inputs in test_loader:
            if inputs is None:
                continue
            inputs = inputs.unsqueeze(1)  # Add channel dimension
            outputs = autoencoder(inputs)
            loss = criterion(outputs, inputs)
            test_loss += loss.item()

        test_loss /= len(test_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Test Loss: {test_loss:.4f}')

    # Save model checkpoint
    checkpoint_path = f'./checkpoints/CNNAutoencoder/model_epoch_{epoch+1}.pth'
    torch.save(autoencoder.state_dict(), checkpoint_path)
    print(f'Model saved to {checkpoint_path}')

Epoch [1/10], Training Loss: 0.1170
Epoch [1/10], Test Loss: 0.5989
Model saved to ./checkpoints/CNNAutoencoder/model_epoch_1.pth
Epoch [2/10], Training Loss: 0.1376
Epoch [2/10], Test Loss: 0.5949
Model saved to ./checkpoints/CNNAutoencoder/model_epoch_2.pth
Epoch [3/10], Training Loss: 0.1480
Epoch [3/10], Test Loss: 0.5939
Model saved to ./checkpoints/CNNAutoencoder/model_epoch_3.pth
Epoch [4/10], Training Loss: 43.7599
Epoch [4/10], Test Loss: 0.5936
Model saved to ./checkpoints/CNNAutoencoder/model_epoch_4.pth
Epoch [5/10], Training Loss: 5.1922
Epoch [5/10], Test Loss: 0.5925
Model saved to ./checkpoints/CNNAutoencoder/model_epoch_5.pth
Epoch [6/10], Training Loss: 0.0489
Epoch [6/10], Test Loss: 0.5875
Model saved to ./checkpoints/CNNAutoencoder/model_epoch_6.pth
Epoch [7/10], Training Loss: 0.1602
Epoch [7/10], Test Loss: 0.5840
Model saved to ./checkpoints/CNNAutoencoder/model_epoch_7.pth
Epoch [8/10], Training Loss: 0.1241
Epoch [8/10], Test Loss: 0.5836
Model saved to ./chec